In [298]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [301]:
# a.u. are used
def psi_b(r, t): # r is supposed to be a vector [x,y,z,m]
    beta = 1
    Eb = - 0.5
    psi = np.sqrt(beta**(1/3))/np.pi * \
          np.multiply(
              np.exp(-beta*r).reshape(r.shape[0],1), \
              np.exp(-1j*Eb*t).reshape(1,t.shape[0]))
    #print(np.exp(-beta*r).shape*np.exp(-1j*Eb*t).shape)
    return psi

In [302]:
x = np.linspace(0,10,128)
t = np.linspace(-10,10,128)
print(psi_b(x,t).shape)
#plt.plot(x,np.abs(psi_b(x,t)))
#plt.plot(x, np.real(psi_b(x,t)))
#plt.plot(x, np.imag(psi_b(x,t)))

(128, 128)


In [305]:
# not ready yet
def psi_b_plus(r,t):
    Ep = 1 # ???
    u_plus = 1 # ???
    c = 0 #integration constant
    a_0 = - e * A0 / wL # A0 should be a vectror
    a_L = a_0*sin(wL*t) # or maybe it is worth to make it a function 
    
    psi = np.exp(
        -1j*Ep*t - \
        i*alpha_L(t)*p - \
        i/2/m * eL**2 * A0**2 *(t + sin(2*wL*t))/2 \
        + c) * \
    u_plus(r)
    return psi

In [306]:
#from scipy.special import hyp1f1
from mpmath import hyp1f1 as hyp1f1
from mpmath import matrix as mp_matrix

In [309]:
#p = [1,2,-3j]
#a = np.linalg.norm(p,2)
def hyper_1F1(r,p): #input just 2 vectors not matrix 
    """it is supposed to return float32 type, but returns mpf"""
    
    Z = 1
    h_bar = 1
    a0 = 1
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2, axis = 0) # is abs equal to norm?
    
    hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # vector
    #result = hyp1f1_fun(1, 1, np.array(z)) 
    hfun = hyp1f1_fun(
        -1j*eta_p,\
        1,\
        -1j*(np.linalg.norm(p,2, axis = 0)*np.linalg.norm(r,2, axis = 0) + np.dot(p,r))/h_bar
    )
    return hfun

In [310]:
#Test
r = np.array([1, 2, 3])
p = np.array([4, 5, 6])
print(hyper_1F1(r,p))

(0.723236263122129 + 0.434200878681639j)


In [311]:
def matrix_hyper_1F1_p(r,p): # we have fixed p here, and go over r vectors
    
    """think about loop removing"""
    #hyp1f1_fun = np.frompyfunc(hyper_1F1, 2, 1)
    #hfun = hyp1f1_fun(r, p)
    values = mp_matrix(1,len(r))
    
    for i in range(len(r)):
        hfun = hyper_1F1(r[i],p)
        values[0,i] = hfun
    return values

In [297]:
#Test
r = np.array([[1, 2, 3], [7, 8, 9]])
p = np.array([7, 8, 9])
print(matrix_hyper_1F1_p(r,p))

[(0.835061926636437 + 0.326741903074577j)  (0.800228898980005 + 0.405875072282826j)]


In [316]:
def matrix_hyp1F1_px(r,p): # p is not fixed anymore
    """think about loop removing"""
    #hyp1f1_fun = np.frompyfunc(hyper_1F1, 2, 1)
    #hfun = hyp1f1_fun(r, p)
    values = mp_matrix(len(p), len(r))
    
    for i in range(len(p)):
        hfun = matrix_hyper_1F1_p(r,p[i])
        values[i,:] = hfun
    return values

    # rows - p changes
    # columns - r changes

In [317]:
#Test
r = np.array([[1, 2, 3], [7, 8, 9]])
p = np.array([[4, 5, 6], [7, 8, 9]])
print(matrix_hyp1F1_px(r,p))

[(0.723236263122129 + 0.434200878681639j)  (0.651149846811972 + 0.539129042554001j)]
[(0.835061926636437 + 0.326741903074577j)  (0.800228898980005 + 0.405875072282826j)]


In [150]:
"""hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # make vector input function 
values = hyp1f1_fun(1, 1, np.array(z)) 
Z = np.array(values.tolist(), dtype = np.float32) # convert mpf type to float32
print(Z)
"""

'hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # make vector input function \nvalues = hyp1f1_fun(1, 1, np.array(z)) \nZ = np.array(values.tolist(), dtype = np.float32) # convert mpf type to float32\nprint(Z)\n'

In [118]:
def columb_wave_plot(r, p): # minus
    Z = 1
    h_bar = 1
    a0 = 1
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2) # is abs equal to norm?
   # print(eta.shape)
    
    hfun = hyp1f1(
        -1j*eta_p,\
        1,\
        -1j*(np.linalg.norm(p,2)*np.linalg.norm(r,2) + np.dot(p,r))/h_bar
    )
    #hfun = hyp1f1(1j,1, r)
    return hfun

In [151]:
p = np.array([1,2,3])
#r = np.array([10, 20, 30])
z = np.linspace(0,1,100)
print(columb_wave_plot(z,p))

TypeError: cannot create mpf from array([ 0.        ,  0.01010101,  0.02020202,  0.03030303,  0.04040404,
        0.05050505,  0.06060606,  0.07070707,  0.08080808,  0.09090909,
        0.1010101 ,  0.11111111,  0.12121212,  0.13131313,  0.14141414,
        0.15151515,  0.16161616,  0.17171717,  0.18181818,  0.19191919,
        0.2020202 ,  0.21212121,  0.22222222,  0.23232323,  0.24242424,
        0.25252525,  0.26262626,  0.27272727,  0.28282828,  0.29292929,
        0.3030303 ,  0.31313131,  0.32323232,  0.33333333,  0.34343434,
        0.35353535,  0.36363636,  0.37373737,  0.38383838,  0.39393939,
        0.4040404 ,  0.41414141,  0.42424242,  0.43434343,  0.44444444,
        0.45454545,  0.46464646,  0.47474747,  0.48484848,  0.49494949,
        0.50505051,  0.51515152,  0.52525253,  0.53535354,  0.54545455,
        0.55555556,  0.56565657,  0.57575758,  0.58585859,  0.5959596 ,
        0.60606061,  0.61616162,  0.62626263,  0.63636364,  0.64646465,
        0.65656566,  0.6666666